In [1]:
import networkx as nx
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
#import plotly.graph_objects as go
import time
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.louvain import louvain_communities
from networkx.algorithms.community.label_propagation import label_propagation_communities
import networkx.algorithms.community as nxcom


In [2]:
df = pd.read_excel(r'cleaned_data.xlsx',sheet_name = 'CD_Filtered')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  461 non-null    int64 
 1   Node_X      461 non-null    object
 2   Node_Y      461 non-null    object
 3   weight      461 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 14.5+ KB


In [4]:
g_full = nx.from_pandas_edgelist(df, source = 'Node_X', target = 'Node_Y', 
                            edge_attr = 'weight',create_using=nx.DiGraph())
#undirected and directed comparison?


In [5]:
len(g_full.nodes)

131

In [6]:
# main_3_chars = ['Harry Potter','Hermione Granger', 'Ron Weasley']

# g_full= G.copy()
# g_full.remove_node(main_3_chars)

In [7]:
len(g_full.nodes)

131

## Undirected Graph

In [8]:
UG = g_full.to_undirected()
for node in g_full:
    for ngbr in nx.neighbors(g_full, node):
        if node in nx.neighbors(g_full, ngbr):
            UG.edges[node, ngbr]['weight'] = (
                g_full.edges[node, ngbr]['weight'] + g_full.edges[ngbr, node]['weight']
            )
#UG.edges.data('weight')
    

## Full Network Community Finding and Visualization

### Set Community to Nodes and Edges

In [9]:
def set_node_community(g, communities):
    """
    :param g: graph
    :param communities: community to which the node belong
    Add community to node attributes
    """
    for i, v_c in enumerate(communities):
        for v in v_c:
            # Add 1 to save 0 for external edges
            g.nodes[v]['community'] = i + 1

def set_edge_community(g):
    """
    :param g: graph
    Find internal edges and add their community to their attributes
    """
    for v, w, in g.edges:
        if g.nodes[v]['community'] == g.nodes[w]['community']:
            # Internal edge, mark with community
            g.edges[v, w]['community'] = g.nodes[v]['community']
        else:
            # External edge, mark as 0
            g.edges[v, w]['community'] = 0

In [10]:
def community_detection_girvan_newman(g):

    result = nxcom.girvan_newman(g)
    communities = next(result)
    print("g_Newman:")
    print(communities)

#     set_node_community(g,communities)
#     set_edge_community(g)

#     df_1 = (pd.DataFrame(list(g.degree), columns=['node','degree']).set_index('node'))
#     df_1['community'] = pd.Series({node:data['community'] for node,data in g.nodes(data=True)})
#     df_1['color'] = df_1.groupby('community')['degree'].transform(lambda c: c/c.max())
#     df_1.loc[df_1['community']=='Officer', 'color'] *= -1

#     plt.figure(figsize=(20, 14))
#     layout = nx.fruchterman_reingold_layout(g)
#     vmin = df_1['color'].min()
#     vmax = df_1['color'].max()
#     cmap = plt.cm.coolwarm

#     nx.draw_networkx(g, pos=layout, with_labels=True, node_color=df_1['color'],cmap=cmap, vmin=vmin, vmax=vmax)
#     sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
#     sm.set_array([])
#     cbar = plt.colorbar(sm)
    #plt.savefig('cluster&degree.png')


In [11]:
def community_detection_modularity(g):

    communities = greedy_modularity_communities(g)
    print("g_modularity:")
    print(communities)

#     set_node_community(g,communities)
#     set_edge_community(g)

#     df_1 = (pd.DataFrame(list(g.degree), columns=['node','degree']).set_index('node'))
#     df_1['community'] = pd.Series({node:data['community'] for node,data in g.nodes(data=True)})
#     df_1['color'] = df_1.groupby('community')['degree'].transform(lambda c: c/c.max())
#     df_1.loc[df_1['community']=='Officer', 'color'] *= -1

#     plt.figure(figsize=(20, 14))
#     layout = nx.fruchterman_reingold_layout(g)
#     vmin = df_1['color'].min()
#     vmax = df_1['color'].max()
#     cmap = plt.cm.coolwarm

#    # nx.draw_networkx(g, pos=layout, with_labels=True, node_color=df_1['color'],cmap=cmap, vmin=vmin, vmax=vmax)
#    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
#    # sm.set_array([])
#    # cbar = plt.colorbar(sm)
#     #plt.savefig('cluster&degree.png')


In [12]:
def community_detection_lovain(g):

    communities = louvain_communities(g)
    print("g_Louvain:")
    print(communities)

#     set_node_community(g,communities)
#     set_edge_community(g)

#     df_1 = (pd.DataFrame(list(g.degree), columns=['node','degree']).set_index('node'))
#     df_1['community'] = pd.Series({node:data['community'] for node,data in g.nodes(data=True)})
#     df_1['color'] = df_1.groupby('community')['degree'].transform(lambda c: c/c.max())
#     df_1.loc[df_1['community']=='Officer', 'color'] *= -1

#     plt.figure(figsize=(20, 14))
#     layout = nx.fruchterman_reingold_layout(g)
#    # vmin = df_1['color'].min()
#     #vmax = df_1['color'].max()
#     cmap = plt.cm.coolwarm

#     #nx.draw_networkx(g, pos=layout, with_labels=True, node_color=df_1['color'],cmap=cmap)#, vmin=vmin, vmax=vmax)
# #     sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
# #     sm.set_array([])
# #     cbar = plt.colorbar(sm)
#     #plt.savefig('cluster&degree.png')


In [13]:
## Undirected Graph
community_detection_modularity(UG)
community_detection_girvan_newman(UG)
community_detection_lovain(UG)

g_modularity:
[frozenset({'Sirius Black', 'Parvati Patil', 'Peter Pettigrew', 'Neville Longbottom', 'Perkins', 'Amos Diggory', 'Arthur Weasley', 'Seamus Finnigan', 'Dean Thomas', 'Nymphadora Tonks', 'Alecto Carrow', 'Sybill Trelawney', 'Cormac McLaggen', 'Kingsley Shacklebolt', 'Goblin', 'Snatcher', 'Cormac Mclaggen', 'Remus Lupin', 'Ginny Weasley', 'Kreacher', 'Nigel Wolpert', 'Walburga Black', 'Rolanda Hooch', 'Molly Weasley', 'Gang thug'}), frozenset({'Filius Flitwick', 'Aragog', 'Everard', 'Quirinus Quirrell', 'Gilderoy Lockhart', 'Firenze', 'Katie Bell', 'Everyone', 'Argus Filch', 'Albus Dumbledore', 'Reubus Hagrid', 'Marcus Belby', 'Rubeus Hagrid', 'Grawp', 'Bane', 'Pomona Sprout', 'Horace Slughorn', 'Dilys Derwent', 'Phineas', 'Olympe Maxime', 'Serpent of Slitherin', 'Tom Riddle', 'Mrs. Cole'}), frozenset({'Arabella Figg', 'Rosmerta', 'Lee Jordan', 'Magorian', 'Cornelius Fudge', 'Tom', 'Headwig', 'Susan Bones', 'Oliver Wood', 'Minerva McGonagall', 'Fang', 'Alastor Moody', 'Mary 

In [14]:
## Directed Graph

community_detection_modularity(g_full)
community_detection_girvan_newman(g_full)
community_detection_lovain(g_full)

g_modularity:
[frozenset({'Percy Weasley', 'Barty Crouch Sr.', 'Arabella Figg', 'Everard', 'Rosmerta', 'Rita Skeeter', 'Cornelius Fudge', 'Nearly Headless Nick', 'Albus Dumbledore', 'Minerva Mcgonagall', 'Reubus Hagrid', 'Barty Crouch Jr.', 'Poppy Pomfrey', 'Viktor Krum', 'Igor Karkaroff', 'Susan Bones', 'Minerva McGonagall', 'Fang', 'Alastor Moody', 'Dilys Derwent', 'The Fat Lady', 'Phineas', 'Cedric Diggory', 'Errol', 'Leanne', 'Mrs. Cole'}), frozenset({'Filius Flitwick', 'Cho Chang', 'Neville Longbottom', 'Katie Bell', 'Everyone', 'Seamus Finnigan', 'Dean Thomas', 'Luna Lovegood', 'Cormac McLaggen', 'Bill Weasley', 'Marcus Belby', 'Snatcher', 'Cormac Mclaggen', 'Fleur Delacour', 'Pomona Sprout', 'Eldred Worple', 'Horace Slughorn', 'Waitress', 'Nigel Wolpert', 'Blaise Zabini', 'Rolanda Hooch', 'Kingsley Shacklebolt', 'Xenophilius Lovegood', 'Tom Riddle'}), frozenset({'Lee Jordan', 'Perkins', 'Arthur Weasley', 'Nymphadora Tonks', 'Amos Diggory', 'Alecto Carrow', 'Goblin', 'Oliver Wood

## To find the importance to reduce the noise

In [15]:
# #deg_centrality
# deg_centrality = nx.degree_centrality(g_full)
# print('character importance of full entity graphs',dict(sorted(deg_centrality.items(), key=lambda kv: (kv[1]), reverse=True)[:10]))
# entities_importance = list(dict(sorted(deg_centrality.items(),key=lambda kv: (kv[1]),reverse=True)).keys())
# print(entities_importance[:20])

# subgraph_entities = entities_importance[:50]
# g_deg = g_full.subgraph(subgraph_entities)

# #betweenness centrality
# deg_centrality_betweenness = nx.betweenness_centrality(g_full, weight='weight')
# print('character importance of full entity graphs',dict(sorted(deg_centrality_betweenness.items(), key=lambda kv: (kv[1]), reverse=True)[:10]))
# entities_importance_betweenness = list(dict(sorted(deg_centrality_betweenness.items(),key=lambda kv: (kv[1]),reverse=True)).keys())
# print(entities_importance_betweenness[:20])

# subgraph_entities = entities_importance_betweenness[:50]
# g_deg_between = g_full.subgraph(subgraph_entities)

# #pagerank data
# deg_centrality_pagerank = nx.pagerank(g_full, weight='weight')
# print('character importance of full entity graphs',dict(sorted(deg_centrality_pagerank.items(), key=lambda kv: (kv[1]), reverse=True)[:10]))
# entities_importance_pagerank = list(dict(sorted(deg_centrality_pagerank.items(),key=lambda kv: (kv[1]),reverse=True)).keys())
# print(entities_importance_pagerank[:20])

# subgraph_entities = entities_importance_pagerank[:50]
# g_pagerank = g_full.subgraph(subgraph_entities)

# #based on k core score
# k_core = list(nx.k_core(g_full))
# g_k_core = g_full.subgraph(k_core)


# # Based on interactions > more than 5
# sub_nodes = []
# for el in g_full.degree():
#     if el[1] > 5:
#         sub_nodes.append(el[0])
# g_interact = g_full.subgraph(sub_nodes)



In [16]:
# #degree centrality 
# community_detection_modularity(g_deg)
# #community_detection_girvan_newman(g_deg)
# community_detection_lovain(g_deg)

In [17]:
# #betweenness centrality 

# community_detection_modularity(g_deg_between)
# #community_detection_girvan_newman(g_deg_between)
# community_detection_lovain(g_deg_between)

In [18]:
# #pagerank centrality 
# community_detection_modularity(g_pagerank)
# #community_detection_girvan_newman(g_pagerank)
# community_detection_lovain(g_pagerank)

In [19]:
# #k core graphs
# community_detection_modularity(g_k_core)
# #community_detection_girvan_newman(g_k_core)
# community_detection_lovain(g_k_core)

In [20]:
# #interactions > 5 graph

# community_detection_modularity(g_interact)
# #community_detection_girvan_newman(g_interact)
# community_detection_lovain(g_interact)